In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
import h5py

import keras
from keras import backend as K
from keras.layers import Input, Dense, Lambda, Layer, Add, Multiply
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.models import model_from_json
import numpy as np

from IPython import display

Using TensorFlow backend.


In [2]:
model_file = './model/vae_decoder.json'
model_weights = './model/vae_decoder.h5'
mu_file = './model/sample_mu.npy'
log_var_file = './model/sample_log_var.npy'

## Loading Model

In [4]:
json_file = open(model_file, 'r')
loaded_model_json = json_file.read()
json_file.close()
decoder = model_from_json(loaded_model_json)

decoder.load_weights(model_weights)

## Loading Meta Data

In [5]:
mu_array = np.load(mu_file)
log_var_array = np.load(log_var_file)

## Relevant Functions

In [11]:
reverse_categorical_map = {0:0, 1:0.5, 2:1, 3:1.5, 4:2, 5:2.5, 6:3, 7:3.5, 8:4, 9:4.5, 10:5}
def sample(model, input_mu, input_log_var, samples_per_z=1):
    multiplied_input_mu = np.repeat(input_mu, samples_per_z, axis=0)
    multiplied_input_log_var = np.repeat(input_log_var, samples_per_z, axis=0)
    eps = np.random.normal(size=(multiplied_input_mu.shape[0], multiplied_input_mu.shape[1]))
    z = reparameterize(multiplied_input_mu, multiplied_input_log_var, eps)
    predictions = model.predict(z, batch_size = None, steps = 1)
    return reconstruct(predictions)
    
def reconstruct(predictions):
    continuous, categorical, binary = predictions
    mu, log_var = np.split(continuous, indices_or_sections = 2,axis = 1)
    eps = np.random.normal(size=mu.shape)
    continuous_data = reparameterize(mu, log_var, eps)
    categorical = np.apply_along_axis(lambda t : np.random.multinomial(1,t), -1, categorical)
    categorical = np.apply_along_axis(lambda t : np.argmax(t), -1, categorical)
    categorical = np.expand_dims(categorical, axis = -1)
    categorical_data = np.apply_along_axis(lambda t : float(reverse_categorical_map[t[0]]), -1, categorical)
    categorical_data = np.expand_dims(categorical_data, axis = -1)
    binary_data = np.apply_along_axis(lambda t: np.random.binomial(1, t), -1, binary)
    coordinates, reviews = np.split(continuous_data, indices_or_sections=[2], axis = 1)
    return np.concatenate([coordinates, categorical_data, reviews, binary_data], axis = 1)

def reparameterize(input_mu, input_log_var, eps):
    sigma = np.exp(0.5*input_log_var)
    return eps*sigma + input_mu

## Sample Generation Done here

In [13]:
vae_samples = sample(decoder, mu_array, log_var_array)
file_name = './samples/vae_1_sample_' + str(len(mu_array))+'_times_'+ str(len(vae_samples)//len(mu_array)) + '_test.csv'
np.savetxt(file_name, vae_samples, delimiter = ',', header='latitude,longitude,stars,review_count,is_open')